In [1]:
import torch
import torch.nn as nn

import numpy as np

import math

class VGG_Schmidtea(nn.Module):
    
        def __init__(self, n_classes=72):
            super(VGG_Schmidtea, self).__init__()
        
            self.conv_1 = nn.Sequential(                                                      # 32 * 32
                nn.Conv2d( 1,   64, kernel_size = 3, padding = 1),  
                nn.Conv2d( 64,  64, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(64),
                nn.MaxPool2d(kernel_size = 2)) 

            self.conv_2 = nn.Sequential(                                                     # 16 * 16 
                nn.Conv2d( 64, 128, kernel_size = 3, padding = 1),   
                nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(128),
                nn.MaxPool2d(kernel_size = 2))

            self.conv_3 = nn.Sequential(                                                     # 8 * 8
                nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
                nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
                nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(256),
                nn.MaxPool2d(kernel_size = 2))

            self.conv_4 = nn.Sequential(                                                     # 4 * 4 
                nn.Conv2d(256, 512, kernel_size = 3, padding = 1),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(512),
                nn.MaxPool2d(kernel_size = 2))

            self.conv_5 = nn.Sequential(                                                     # 2 * 2
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(512),
                nn.MaxPool2d(kernel_size = 2))                                               # 1 * 1 


            self.classifier = nn.Sequential(
                nn.Dropout(p = 0.25),
                nn.Linear(512, 4096),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(4096),
                nn.Dropout(p = 0.25),
                nn.Linear(4096, 4096),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(4096),
                nn.Linear(4096, n_classes))

             
        def forward(self, x):
            x = self.conv_1(x)
            x = self.conv_2(x)
            x = self.conv_3(x)
            x = self.conv_4(x)
            x = self.conv_5(x)
            x = torch.flatten(x, 1)
            x = self.classifier(x)
                    
            return x

In [2]:
class VGG_Schmidtea(nn.Module):
    
        def __init__(self, n_classes=72):
            super(VGG_Schmidtea, self).__init__()
        
            self.conv_1 = nn.Sequential(                                                      # 32 * 32
                nn.Conv2d( 1,   64, kernel_size = 3, padding = 1), 
                nn.ReLU(inplace=True),
                nn.Conv2d( 64,  64, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(64),
                nn.MaxPool2d(kernel_size = 2)) 

            self.conv_2 = nn.Sequential(                                                     # 16 * 16 
                nn.Conv2d( 64, 128, kernel_size = 3, padding = 1), 
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(128),
                nn.MaxPool2d(kernel_size = 2))

            self.conv_3 = nn.Sequential(                                                     # 8 * 8
                nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(256),
                nn.MaxPool2d(kernel_size = 2))

            self.conv_4 = nn.Sequential(                                                     # 4 * 4 
                nn.Conv2d(256, 512, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(512),
                nn.MaxPool2d(kernel_size = 2))

            self.conv_5 = nn.Sequential(                                                 # 2 * 2
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(512),
                nn.MaxPool2d(kernel_size = 2))                                               # 1 * 1 


            self.classifier = nn.Sequential(
                nn.Dropout(p = 0.25),
                nn.Linear(512, 4096),
                #nn.ReLU(inplace=True),
                nn.Hardtanh(-1, 1), 
                nn.BatchNorm1d(4096),
                nn.Dropout(p = 0.25),
                nn.Linear(4096, 4096),
                #nn.ReLU(inplace=True),
                nn.Hardtanh(-1, 1), 
                #nn.BatchNorm1d(4096),
                nn.Linear(4096, n_classes))

             
        def forward(self, x):
            x = self.conv_1(x)
            x = self.conv_2(x)
            x = self.conv_3(x)
            x = self.conv_4(x)
            x = self.conv_5(x)
            x = torch.flatten(x, 1)
            x = self.classifier(x)
                    
            return x

In [3]:
def train_(model,train_loader, device, problem, criterion, optimizer, epoch, n_class, loss_vector, accuracy_vector):
    # Set model to training mode
    model.train()

    train_loss, correct = 0, 0
        
    # Loop over each batch from the training set
    for batch_idx, batch in enumerate(train_loader):
        
        # Send data to the device (GPU) in the appropriate format
        img = batch['image'].float().to(device)
        angle = batch['angle'].float().to(device)        

        # Zero gradient buffers
        optimizer.zero_grad()
        
        # Pass data through the network
        output = model(img)
        
        cosangle = torch.cos(torch.deg2rad(angle))
        sinangle = torch.sin(torch.deg2rad(angle))
        tangle = torch.stack([sinangle, cosangle]).view((len(angle), 2))
        
        loss = criterion(output, tangle)

        train_loss += loss
  
        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        ## Calcul of correct predicted angle
        real_angle = torch.rad2deg(torch.atan2(tangle[:,1], tangle[:,0]))
        pred_angle = torch.rad2deg(torch.atan2(output[:,1], output[:,0]))

        angle_5 = real_angle//5
        pred_5 = pred_angle//5
        correct += pred_5.eq(angle_5).cpu().sum()
            
    
    train_loss /= len(train_loader)
    loss_vector.append(train_loss.detach().cpu().numpy())
    
    accuracy = correct.to(torch.float32)/len(train_loader.dataset)*100
    accuracy_vector.append(accuracy.cpu().numpy())
    
    print(f" Training: Loss: {train_loss:.3f}, Accuracy: {accuracy:.1f}    |    ", end = '')


In [3]:
  
def validate_(model, validation_loader, device, problem, criterion, n_class, loss_vector, accuracy_vector):
    '''
    Input of the function:
        model: neural network model in Pytorch
        loss_vector: empty array with is assigned by the function
        accuracy_vector: empty array with is assigned by the function
    '''

    val_loss, correct = 0, 0
        
    i = 0
    for batch_idx, batch in enumerate(validation_loader):
        i +=1
        # Copy data to GPU if needed
        img = batch['image'].float().to(device)
        angle = batch['angle'].float().to(device)        
        
        # Pass data through the network
        with torch.no_grad():
                    
            # Pass data through the network
            output = model(img)

            cosangle = torch.cos(torch.deg2rad(angle))
            sinangle = torch.sin(torch.deg2rad(angle))
            tangle = torch.stack([sinangle, cosangle]).view((len(angle), 2))

            loss = criterion(output, tangle)

            val_loss += loss
    
            real_angle = torch.rad2deg(torch.atan2(tangle[:,1], tangle[:,0]))
            pred_angle = torch.rad2deg(torch.atan2(output[:,1], output[:,0]))
            
            angle_5 = real_angle//5
            pred_5 = pred_angle//5
            correct += pred_5.eq(angle_5).cpu().sum()
            
            #angle_list.append(real_angle.detach().flatten().numpy())
            #pred_list.append(real_angle.detach().flatten().numpy())
            
    
    val_loss /= len(validation_loader)
    loss_vector.append(val_loss.detach().cpu().numpy())
    
    accuracy = correct.to(torch.float32)/len(validation_loader.dataset)*100
    accuracy_vector.append(accuracy.detach().cpu().numpy())
    
    print(f" Validation: Loss: {val_loss:.3f}, Accuracy: {accuracy:.1f}")

In [1]:
def predictor(model, a_centriole, device, problem = 'classification'):
    model.eval()
    
    img = torch.from_numpy(a_centriole)
    img = img.float().to(device)
    output = model(img)
    
    if problem == 'classification':
        pred = output.max(1)[1]
    else:
        pred = output
        
    return pred   